In [6]:
from captcha.image import ImageCaptcha
import cv2
import random
import string
from keras.models import *
from keras.layers import *
from keras import optimizers
from keras import losses
from keras.models import load_model
import glob

In [7]:
def generate_dataset(environment, n):
    image = ImageCaptcha()
    for i in range(n):
        rnd = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
        rnd += str(i)
        image.write(rnd[:4], environment + '_dataset_full/' + rnd + '.png')

In [21]:
generate_dataset("train", 5000)

In [33]:
generate_dataset("test", 1000)

In [8]:
def load_dataset(environment):
    filenames = glob.glob(environment + "_dataset_full/*.png")
    x1 = []
    x2 = []
    x3 = []
    x4 = []
    y1 = []
    y2 = []
    y3 = []
    y4 = []
    for img in filenames:
        tmp = cv2.imread(img)
        # uzeo samo pocetak slike
        p1 = tmp[0:60, 0:69]
        p2 = tmp[0:60, 20:89]
        p3 = tmp[0:60, 40:109]
        p4 = tmp[0:60, 60:129]
        x1.append(p1)
        x2.append(p2)
        x3.append(p3)
        x4.append(p4)
        q1 = ord(img[len(environment)+14])-65
        q2 = ord(img[len(environment)+15])-65
        q3 = ord(img[len(environment)+16])-65
        q4 = ord(img[len(environment)+17])-65
        y1.append(q1)
        y2.append(q2)
        y3.append(q3)
        y4.append(q4)
    return np.array(x1), np.array(x2), np.array(x3), np.array(x4), np.array(y1), np.array(y2), np.array(y3), np.array(y4), 26

In [9]:
x1_train, x2_train, x3_train, x4_train, y1_train, y2_train, y3_train, y4_train, C = load_dataset("train")

In [10]:
x1_test, x2_test, x3_test, x4_test, y1_test, y2_test, y3_test, y4_test, C = load_dataset("test")

In [11]:
def threshold_data(img_data, threshold):
    for i in range(len(img_data)):
        for j in range(len(img_data[0])):
            for k in range(len(img_data[0][0])):
                for l in range(3):
                    if img_data[i][j][k][l] > threshold:
                        img_data[i][j][k][l] = 0 # black
                    else:
                        img_data[i][j][k][l] = 255 # max value of channel
    return img_data

In [12]:
x1_train = threshold_data(x1_train, 150)

In [13]:
x2_train = threshold_data(x2_train, 150)

In [14]:
x3_train = threshold_data(x3_train, 150)

In [15]:
x4_train = threshold_data(x4_train, 150)

In [16]:
x1_test = threshold_data(x1_test, 150)

In [17]:
x2_test = threshold_data(x2_test, 150)

In [18]:
x3_test = threshold_data(x3_test, 150)

In [19]:
x4_test = threshold_data(x4_test, 150)

In [20]:
def one_hot(y, C):
    y_one_hot = np.zeros((y.shape[0], C))
    y_one_hot[np.arange(0, y.shape[0]), y] = 1
    
    return y_one_hot

In [21]:
x1_train = x1_train/255
x2_train = x2_train/255
x3_train = x3_train/255
x4_train = x4_train/255

In [22]:
x1_test = x1_test/255
x2_test = x2_test/255
x3_test = x3_test/255
x4_test = x4_test/255

In [23]:
y1_train = one_hot(y1_train, C)
y2_train = one_hot(y2_train, C)
y3_train = one_hot(y3_train, C)
y4_train = one_hot(y4_train, C)

In [24]:
y1_test = one_hot(y1_test, C)
y2_test = one_hot(y2_test, C)
y3_test = one_hot(y3_test, C)
y4_test = one_hot(y4_test, C)

In [25]:
def build_model(image_shape, c):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=image_shape))
    model.add(Convolution2D(20, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(40, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(80, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(40, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())

    model.add(Dense(1000, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(c, activation='softmax'))
    
    return model

In [26]:
model1 = build_model(x1_train[0].shape, C)

D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(40, (3, 3), activation="relu")`
  
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(80, (3, 3), activation="relu")`
  if sys.path[0] == '':
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(40, (3, 3), activation="relu")`
  app.launch_new_instance()


In [27]:
model2 = build_model(x2_train[0].shape, C)

D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(40, (3, 3), activation="relu")`
  
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(80, (3, 3), activation="relu")`
  if sys.path[0] == '':
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(40, (3, 3), activation="relu")`
  app.launch_new_instance()


In [28]:
model3 = build_model(x3_train[0].shape, C)

D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(40, (3, 3), activation="relu")`
  
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(80, (3, 3), activation="relu")`
  if sys.path[0] == '':
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(40, (3, 3), activation="relu")`
  app.launch_new_instance()


In [29]:
model4 = build_model(x4_train[0].shape, C)

D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(40, (3, 3), activation="relu")`
  
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(80, (3, 3), activation="relu")`
  if sys.path[0] == '':
D:\ProgramData\Anaconda2\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(40, (3, 3), activation="relu")`
  app.launch_new_instance()


In [30]:
model1.compile(loss=losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [31]:
model2.compile(loss=losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [32]:
model3.compile(loss=losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [33]:
model4.compile(loss=losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [34]:
history1 = model1.fit(x1_train, y1_train, verbose=1, epochs=15, batch_size=32, shuffle=True)

Epoch 1/15
5000/5000 [==============================] - 143s 29ms/step - loss: 2.8877 - acc: 0.1530
Epoch 2/15
5000/5000 [==============================] - 140s 28ms/step - loss: 1.3972 - acc: 0.5552
Epoch 3/15
5000/5000 [==============================] - 141s 28ms/step - loss: 0.8775 - acc: 0.7198
Epoch 4/15
5000/5000 [==============================] - 140s 28ms/step - loss: 0.6943 - acc: 0.7878
Epoch 5/15
5000/5000 [==============================] - 146s 29ms/step - loss: 0.5754 - acc: 0.8250
Epoch 6/15
5000/5000 [==============================] - 148s 30ms/step - loss: 0.4354 - acc: 0.8648
Epoch 7/15
5000/5000 [==============================] - 146s 29ms/step - loss: 0.2801 - acc: 0.9136
Epoch 8/15
5000/5000 [==============================] - 145s 29ms/step - loss: 0.1763 - acc: 0.9442
Epoch 9/15
5000/5000 [==============================] - 158s 32ms/step - loss: 0.1319 - acc: 0.9574
Epoch 10/15
5000/5000 [==============================] - 147s 29ms/step - loss: 0.0867 - acc: 0.9772

In [35]:
history2 = model2.fit(x2_train, y2_train, verbose=1, epochs=15, batch_size=32, shuffle=True)

Epoch 1/15
5000/5000 [==============================] - 141s 28ms/step - loss: 3.2562 - acc: 0.0418
Epoch 2/15
5000/5000 [==============================] - 141s 28ms/step - loss: 2.6367 - acc: 0.2084
Epoch 3/15
5000/5000 [==============================] - 146s 29ms/step - loss: 1.7860 - acc: 0.4354
Epoch 4/15
5000/5000 [==============================] - 143s 29ms/step - loss: 1.2943 - acc: 0.5812
Epoch 5/15
5000/5000 [==============================] - 151s 30ms/step - loss: 1.0054 - acc: 0.6842
Epoch 6/15
5000/5000 [==============================] - 153s 31ms/step - loss: 0.8019 - acc: 0.7474
Epoch 7/15
5000/5000 [==============================] - 152s 30ms/step - loss: 0.6305 - acc: 0.7986
Epoch 8/15
5000/5000 [==============================] - 152s 30ms/step - loss: 0.4823 - acc: 0.8470
Epoch 9/15
5000/5000 [==============================] - 151s 30ms/step - loss: 0.3111 - acc: 0.9012
Epoch 10/15
5000/5000 [==============================] - 150s 30ms/step - loss: 0.2124 - acc: 0.9288

In [36]:
history3 = model3.fit(x3_train, y3_train, verbose=1, epochs=15, batch_size=32, shuffle=True)

Epoch 1/15
5000/5000 [==============================] - 141s 28ms/step - loss: 3.2584 - acc: 0.0374
Epoch 2/15
5000/5000 [==============================] - 140s 28ms/step - loss: 2.7653 - acc: 0.1858
Epoch 3/15
5000/5000 [==============================] - 140s 28ms/step - loss: 1.8685 - acc: 0.4300
Epoch 4/15
5000/5000 [==============================] - 141s 28ms/step - loss: 1.3752 - acc: 0.5780
Epoch 5/15
5000/5000 [==============================] - 147s 29ms/step - loss: 1.0807 - acc: 0.6714
Epoch 6/15
5000/5000 [==============================] - 153s 31ms/step - loss: 0.8826 - acc: 0.7286
Epoch 7/15
5000/5000 [==============================] - 156s 31ms/step - loss: 0.6876 - acc: 0.7828
Epoch 8/15
5000/5000 [==============================] - 154s 31ms/step - loss: 0.5727 - acc: 0.8212
Epoch 9/15
5000/5000 [==============================] - 154s 31ms/step - loss: 0.4412 - acc: 0.8600
Epoch 10/15
5000/5000 [==============================] - 155s 31ms/step - loss: 0.3127 - acc: 0.9062

In [37]:
history4 = model4.fit(x4_train, y4_train, verbose=1, epochs=15, batch_size=32, shuffle=True)

Epoch 1/15
5000/5000 [==============================] - 146s 29ms/step - loss: 3.2590 - acc: 0.0432
Epoch 2/15
5000/5000 [==============================] - 143s 29ms/step - loss: 3.1305 - acc: 0.0778
Epoch 3/15
5000/5000 [==============================] - 144s 29ms/step - loss: 2.2824 - acc: 0.3080
Epoch 4/15
5000/5000 [==============================] - 144s 29ms/step - loss: 1.4354 - acc: 0.5568
Epoch 5/15
5000/5000 [==============================] - 153s 31ms/step - loss: 1.0200 - acc: 0.6806
Epoch 6/15
5000/5000 [==============================] - 159s 32ms/step - loss: 0.8196 - acc: 0.7438
Epoch 7/15
5000/5000 [==============================] - 161s 32ms/step - loss: 0.7195 - acc: 0.7798
Epoch 8/15
5000/5000 [==============================] - 159s 32ms/step - loss: 0.6907 - acc: 0.7860
Epoch 9/15
5000/5000 [==============================] - 160s 32ms/step - loss: 0.6334 - acc: 0.8074
Epoch 10/15
5000/5000 [==============================] - 158s 32ms/step - loss: 0.6129 - acc: 0.8140

In [38]:
model1.evaluate(x1_test, y1_test)

1000/1000 [==============================] - 9s 9ms/step


[1.7870943059921265, 0.775]

In [39]:
model2.evaluate(x2_test, y2_test)

1000/1000 [==============================] - 9s 9ms/step


[2.6533738422393798, 0.646]

In [40]:
model3.evaluate(x3_test, y3_test)

1000/1000 [==============================] - 9s 9ms/step


[2.771801127433777, 0.606]

In [41]:
model4.evaluate(x4_test, y4_test)

1000/1000 [==============================] - 9s 9ms/step


[1.0798555126190186, 0.731]

In [97]:
def test_captcha(text, model1, model2, model3, model4):
    if len(text) != 4:
        return 'CAPTCHA must have 4 letters'
    image = ImageCaptcha()
    image.write(text, 'out.png')
    y1 = text[0]
    y2 = text[1]
    y3 = text[2]
    y4 = text[3]
    tmp = cv2.imread('out.png')
    x1 = threshold_data(np.array([tmp[0:60, 0:69]]), 150) / 255
    x2 = threshold_data(np.array([tmp[0:60, 20:89]]), 150) / 255
    x3 = threshold_data(np.array([tmp[0:60, 40:109]]), 150) / 255
    x4 = threshold_data(np.array([tmp[0:60, 60:129]]), 150) / 255
    y1_predicted = np.argmax(model1.predict(x1, verbose=1))
    y2_predicted = np.argmax(model2.predict(x2, verbose=1))
    y3_predicted = np.argmax(model3.predict(x3, verbose=1))
    y4_predicted = np.argmax(model4.predict(x4, verbose=1))
    print("Predicted: " + chr(y1_predicted+65) + chr(y2_predicted+65) + chr(y3_predicted+65) + chr(y4_predicted+65))

In [105]:
model1.save('captcha1.h5')
model2.save('captcha2.h5')
model3.save('captcha3.h5')
model4.save('captcha4.h5')

In [104]:
test_captcha('QFEA', model1, model2, model3, model4)

1/1 [==============================] - 0s 16ms/step
Predicted: QFEA


In [ ]:
model1 = load_model('captcha1.h5')
model2 = load_model('captcha2.h5')
model3 = load_model('captcha3.h5')
model4 = load_model('captcha4.h5')